In [1]:
# ========================
# 0. IMPORTS
# ========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, RepeatVector, TimeDistributed,
                                    MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D,
                                    Conv1D)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib
import os

# ========================
# 1. CONFIGURATION
# ========================
FORECAST_STEPS = 10  # Window size for autoencoder
THRESHOLD_PERCENTILE = 90
# Transformer parameters for anomaly detection
EMBED_DIM = 128
NUM_HEADS = 4
FF_DIM = 256
DROPOUT_RATE = 0.1
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ========================
# 2. DEVICE SETUP
# ========================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU is available and will be used.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")

# ========================
# 3. LOAD AND PREPROCESS DATA
# ========================
# Load original labeled dataset and parse DateTime column
original_df = pd.read_csv('../../data/test_set.csv')
original_df['DateTime'] = pd.to_datetime(original_df['DateTime'], errors='coerce')
original_df.set_index('DateTime', inplace=True)

# Load cleaned labeled dataset and parse DateTime column
cleaned_df = pd.read_csv('../../data/cleaned_labeled_dataset.csv')
cleaned_df['DateTime'] = pd.to_datetime(cleaned_df['DateTime'], errors='coerce')
cleaned_df.set_index('DateTime', inplace=True)

# Separate out labels for evaluation
original_labels = original_df['labels'].copy()
original_df.drop(columns=['labels'], inplace=True)
cleaned_df.drop(columns=['labels'], inplace=True)

# Drop columns with more than 30% missing values and fill the rest
original_df.dropna(axis=1, thresh=int(0.7 * len(original_df)), inplace=True)
original_df.ffill(inplace=True)  # Forward fill
original_df.bfill(inplace=True)  # Backward fill

cleaned_df.dropna(axis=1, thresh=int(0.7 * len(cleaned_df)), inplace=True)
cleaned_df.ffill(inplace=True)
cleaned_df.bfill(inplace=True)

# Normalize data using MinMaxScaler to scale features to [0, 1]
scaler = MinMaxScaler()
original_scaled = scaler.fit_transform(original_df.values)  # Fit on original data
cleaned_scaled = scaler.transform(cleaned_df.values)  # Transform cleaned data using the same scaler

# Convert normalized arrays back to DataFrames, preserving index and column names
original_df_scaled = pd.DataFrame(original_scaled, index=original_df.index, columns=original_df.columns).astype(np.float32)
cleaned_df_scaled = pd.DataFrame(cleaned_scaled, index=cleaned_df.index, columns=cleaned_df.columns).astype(np.float32)

print(f"✅ Original dataset shape: {original_df_scaled.shape}")
print(f"✅ Cleaned dataset shape: {cleaned_df_scaled.shape}")

# ========================
# 4. CREATE SEQUENCES FOR AUTOENCODER
# ========================
def create_ae_sequences(data, seq_len):
    """Create fixed-length sequences for autoencoder (input == output)"""
    return np.array([data[i:i+seq_len] for i in range(len(data) - seq_len)], dtype=np.float32)

# Create sequences for training the autoencoder from cleaned data
X_ae_train = create_ae_sequences(cleaned_df_scaled.values, FORECAST_STEPS)
print(f"✅ Training sequences shape: {X_ae_train.shape}")

# ========================
# 5. BUILD TRANSFORMER AUTOENCODER
# ========================
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
    """Transformer encoder block"""
    # Multi-head self-attention
    attention_output = MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(inputs + attention_output)
    
    # Feed-forward network
    ffn_output = Dense(ff_dim, activation="relu")(attention_output)
    ffn_output = Dense(embed_dim)(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    
    # Add & Norm
    return LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)

def build_transformer_autoencoder(input_steps, input_dim, embed_dim=128, num_heads=4, ff_dim=256, dropout_rate=0.1):
    """Build a transformer-based autoencoder for anomaly detection"""
    inputs = Input(shape=(input_steps, input_dim))
    
    # Initial projection to embed_dim
    x = Conv1D(filters=embed_dim, kernel_size=1, activation='relu')(inputs)
    
    # Encoder: Transformer blocks
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    
    # Bottleneck
    encoded = GlobalAveragePooling1D()(x)
    
    # Decoder: Expand to sequence
    x = RepeatVector(input_steps)(encoded)
    
    # Decoder: Transformer blocks
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    
    # Output projection back to original dimensions
    outputs = TimeDistributed(Dense(input_dim))(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# ========================
# 6. TRAIN TRANSFORMER AUTOENCODER
# ========================
# Build the transformer autoencoder
transformer_ae = build_transformer_autoencoder(
    FORECAST_STEPS, 
    X_ae_train.shape[2],
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    dropout_rate=DROPOUT_RATE
)
transformer_ae.summary()

# Train with early stopping
es = EarlyStopping(patience=5, restore_best_weights=True)
history = transformer_ae.fit(
    X_ae_train, 
    X_ae_train, 
    validation_split=0.1, 
    epochs=20, 
    batch_size=64, 
    callbacks=[es], 
    verbose=1
)

# Save model

print("\n✅ Transformer Autoencoder trained and saved.")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Transformer Autoencoder Training History')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.savefig("transformer_ae_training_history.png")
plt.close()

# ========================
# 7. EVALUATION ON TEST DATA
# ========================
# Use the last 3000 entries from the original dataset for testing
test_data = original_df_scaled.tail(3000).reset_index(drop=True)
test_labels = original_labels.tail(3000).reset_index(drop=True)
print(f"✅ Test data shape: {test_data.shape}")

# Create sequences for testing
test_sequences = create_ae_sequences(test_data.values, FORECAST_STEPS)
print(f"✅ Test sequences shape: {test_sequences.shape}")

# Process all test sequences in batches for memory efficiency
reconstruction_errors = []
batch_size = 128
for i in range(0, len(test_sequences), batch_size):
    batch = test_sequences[i:min(i+batch_size, len(test_sequences))]
    # Get reconstructions
    reconstructions = transformer_ae.predict(batch, verbose=0)
    # Calculate reconstruction error for each sequence
    for j in range(len(batch)):
        original = batch[j]
        reconstructed = reconstructions[j]
        # Calculate MSE for each timestep across all features
        error_per_timestep = np.mean((original - reconstructed)**2, axis=1)
        reconstruction_errors.append(error_per_timestep)

# Convert to numpy array for easier manipulation
reconstruction_errors = np.array(reconstruction_errors)

# Calculate error scores: aggregate multiple predictions for each timestep
timesteps_total = len(test_data) - FORECAST_STEPS + 1  # Number of sequences
aggregated_errors = np.zeros(timesteps_total + FORECAST_STEPS - 1)  # Total timesteps covered
counts = np.zeros(timesteps_total + FORECAST_STEPS - 1)  # Count how many predictions for each timestep

# For each sequence
for i in range(len(reconstruction_errors)):
    # For each timestep in the sequence
    for j in range(FORECAST_STEPS):
        # Add error to the corresponding position in the original time series
        aggregated_errors[i + j] += reconstruction_errors[i][j]
        counts[i + j] += 1

# Average the errors where we have multiple predictions
aggregated_errors = np.divide(aggregated_errors, counts, where=counts>0)

# Calculate threshold
threshold = np.percentile(aggregated_errors[counts > 0], THRESHOLD_PERCENTILE)

# Generate anomaly flags
anomaly_flags = (aggregated_errors > threshold).astype(int)

# Align with original labels
# The labels need to be aligned with the outputs of our autoencoder
aligned_labels = test_labels.values[:len(anomaly_flags)]

# Make sure both arrays are the same length
min_len = min(len(anomaly_flags), len(aligned_labels))
anomaly_flags = anomaly_flags[:min_len]
aligned_labels = aligned_labels[:min_len]

# ========================
# 8. COMPUTE METRICS
# ========================
precision = precision_score(aligned_labels, anomaly_flags, zero_division=0)
recall = recall_score(aligned_labels, anomaly_flags, zero_division=0)
f1 = f1_score(aligned_labels, anomaly_flags, zero_division=0)

print("\nAnomaly Detection Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Calculate confusion matrix values for detailed analysis
true_positives = ((aligned_labels == 1) & (anomaly_flags == 1)).sum()
false_positives = ((aligned_labels == 0) & (anomaly_flags == 1)).sum()
true_negatives = ((aligned_labels == 0) & (anomaly_flags == 0)).sum()
false_negatives = ((aligned_labels == 1) & (anomaly_flags == 0)).sum()

print("\nConfusion Matrix Details:")
print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives: {true_negatives}")
print(f"False Negatives: {false_negatives}")
print(f"Total Anomaly Timesteps in Ground Truth: {aligned_labels.sum()}")
print(f"Total Timesteps Flagged as Anomalies: {anomaly_flags.sum()}")

# Save metrics to CSV
metrics_results = {
    "Model": "Transformer Autoencoder",
    "Precision": precision,
    "Recall": recall,
    "F1_Score": f1,
    "True_Positives": true_positives,
    "False_Positives": false_positives,
    "True_Negatives": true_negatives,
    "False_Negatives": false_negatives,
    "Anomaly_Count_True": aligned_labels.sum(),
    "Anomaly_Count_Predicted": anomaly_flags.sum()
}

metrics_df = pd.DataFrame([metrics_results])
metrics_df.to_csv("transformer_ae_anomaly_detection_metrics.csv", index=False)
print("\n✅ Metrics saved to 'transformer_ae_anomaly_detection_metrics.csv'.")

# ========================
# 9. VISUALIZATION
# ========================
# Plot 1: Reconstruction Error and Anomalies
plt.figure(figsize=(15, 10))

plt.subplot(3, 1, 1)
plt.plot(aggregated_errors, label='Reconstruction Error', alpha=0.7)
plt.axhline(threshold, color='r', linestyle='--', label=f'Threshold ({THRESHOLD_PERCENTILE}th percentile)')
plt.title('Reconstruction Error with Anomaly Detection')
plt.ylabel('Error')
plt.legend()
plt.grid(True)

# Plot 2: Detected Anomalies
plt.subplot(3, 1, 2)
plt.scatter(
    np.where(anomaly_flags == 1)[0], 
    aggregated_errors[anomaly_flags == 1], 
    color='red', 
    label='Detected Anomalies', 
    s=20
)
plt.axhline(threshold, color='r', linestyle='--', label=f'Threshold ({THRESHOLD_PERCENTILE}th percentile)')
plt.title('Detected Anomalies')
plt.ylabel('Error')
plt.legend()
plt.grid(True)

# Plot 3: Comparing with Ground Truth
plt.subplot(3, 1, 3)
plt.plot(aligned_labels, label='True Anomalies', color='blue', alpha=0.5)
plt.plot(anomaly_flags, label='Predicted Anomalies', color='red', alpha=0.5)
plt.title('Ground Truth vs Predicted Anomalies')
plt.xlabel('Timestep')
plt.ylabel('Anomaly (1=Yes, 0=No)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("transformer_ae_anomaly_detection_results.png")
plt.close()

# Additional plot to visualize matches and mismatches
plt.figure(figsize=(15, 5))
anomaly_comparison = np.zeros(len(aligned_labels))
anomaly_comparison[(aligned_labels == 1) & (anomaly_flags == 1)] = 3  # True Positives
anomaly_comparison[(aligned_labels == 0) & (anomaly_flags == 1)] = 2  # False Positives
anomaly_comparison[(aligned_labels == 1) & (anomaly_flags == 0)] = 1  # False Negatives
# True Negatives are left as 0

plt.plot(anomaly_comparison, 'o-', markersize=2)
plt.grid(True)
plt.yticks([0, 1, 2, 3], ['True Negative', 'False Negative', 'False Positive', 'True Positive'])
plt.title('Anomaly Detection Performance Visualization')
plt.xlabel('Timestep')
plt.savefig("transformer_ae_detection_performance.png")
plt.close()

# Plot distribution of reconstruction errors
plt.figure(figsize=(12, 6))
plt.hist(aggregated_errors, bins=100, alpha=0.7)
plt.axvline(threshold, color='r', linestyle='--', label=f'Threshold ({THRESHOLD_PERCENTILE}th percentile)')
plt.title('Distribution of Reconstruction Errors')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.savefig("transformer_ae_error_distribution.png")
plt.close()

print("\n✅ All visualizations saved.")

# Optional: Try different threshold values to find optimal F1 score
print("\nThreshold Tuning:")
best_f1 = 0
best_threshold = threshold
best_precision = precision
best_recall = recall

thresholds = np.percentile(aggregated_errors[counts > 0], [80, 85, 90, 95, 97, 99])
for potential_threshold in thresholds:
    potential_flags = (aggregated_errors > potential_threshold).astype(int)
    potential_precision = precision_score(aligned_labels[:len(potential_flags)], potential_flags, zero_division=0)
    potential_recall = recall_score(aligned_labels[:len(potential_flags)], potential_flags, zero_division=0)
    potential_f1 = f1_score(aligned_labels[:len(potential_flags)], potential_flags, zero_division=0)
    
    print(f"Threshold percentile: {np.round(100 * len(aggregated_errors[aggregated_errors > potential_threshold]) / len(aggregated_errors), 2)}%")
    print(f"Precision: {potential_precision:.4f}, Recall: {potential_recall:.4f}, F1: {potential_f1:.4f}")
    
    if potential_f1 > best_f1:
        best_f1 = potential_f1
        best_threshold = potential_threshold
        best_precision = potential_precision
        best_recall = potential_recall

print(f"\nBest Threshold Found: {best_threshold:.6f}")
print(f"Best Precision: {best_precision:.4f}")
print(f"Best Recall: {best_recall:.4f}")
print(f"Best F1 Score: {best_f1:.4f}")

✅ GPU is available and will be used.
✅ Original dataset shape: (12131, 26)
✅ Cleaned dataset shape: (18652, 26)
✅ Training sequences shape: (18642, 10, 26)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 26)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 10, 128)      3456        ['input_1[0][0]']                
                                                                                                  
 multi_head_attention (MultiHea  (None, 10, 128)     263808      ['conv1d[0][0]',                 
 dAttention)                                                      'conv1d[0][0]']                 
                                     

In [ ]:
THRESHOLD_PERCENTILE = 99.9999

# ========================
# 9. REAL-TIME SIMULATION ON TEST SET
# ========================
# Import necessary libraries for ROC curve analysis
from sklearn.metrics import roc_curve, auc, precision_recall_curve

simulation_X, simulation_y = create_sequences(test_data_tail.values, INPUT_STEPS, FORECAST_STEPS)
# Create corresponding labels for evaluation
simulation_labels = create_sequence_labels(test_labels_tail.values, INPUT_STEPS, FORECAST_STEPS)

forecast_list = []
reconstruction_list = []
reconstruction_errors = []
true_windows = []

# Process all samples without overlapping
step_size = WINDOW_SIZE_SIMULATION  # Use the window size as step size to avoid overlap
for i in range(0, len(simulation_X), step_size):
    # Get the current window batch (up to step_size samples)
    window_X = simulation_X[i:i+step_size]
    window_y_true = simulation_y[i:i+step_size]
    
    if len(window_X) == 0:
        continue
    
    # Use LSTM Seq2Seq for forecasting
    y_pred_future = best_model.predict(window_X, batch_size=128, verbose=1)
    X_forecast = y_pred_future  # No need to expand dims as we're processing batches
    
    # Use Transformer Autoencoder for anomaly detection
    y_reconstructed = transformer_ae.predict(X_forecast, batch_size=128, verbose=1)
    
    # Calculate reconstruction error for each sample in the batch
    batch_reconstruction_errors = np.mean((y_pred_future - y_reconstructed)**2, axis=(1, 2))
    
    # Store predictions and errors
    forecast_list.append(y_pred_future)
    reconstruction_list.append(y_reconstructed)
    reconstruction_errors.append(batch_reconstruction_errors)
    true_windows.append(window_y_true)

print("\n✅ Real-time simulation complete.")

# Flatten the reconstruction errors list for threshold calculation
all_reconstruction_errors = np.concatenate(reconstruction_errors)

# Make sure we're only considering windows where we have both predictions and labels
min_length = min(len(simulation_labels), len(all_reconstruction_errors))
true_labels_subset = simulation_labels[:min_length]
errors_subset = all_reconstruction_errors[:min_length]

# Find optimal threshold using ROC curve analysis
def find_optimal_threshold_roc(y_true, reconstruction_errors):
    """
    Find the optimal threshold using ROC curve analysis.
    Returns the threshold that maximizes Youden's J statistic (TPR - FPR)
    """
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, reconstruction_errors)
    
    # Calculate AUC
    roc_auc = auc(fpr, tpr)
    
    # Find the threshold that maximizes TPR - FPR (Youden's J statistic)
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    # Plot ROC curve
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, 'b-', label=f'ROC curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
    plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='red', 
                label=f'Optimal threshold: {optimal_threshold:.5f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Anomaly Detection')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.savefig('roc_curve_threshold.png')
    
    print(f"✅ ROC analysis - Optimal threshold: {optimal_threshold:.5f}")
    print(f"✅ ROC analysis - AUC: {roc_auc:.5f}")
    print(f"✅ ROC analysis - At optimal threshold: TPR={tpr[optimal_idx]:.5f}, FPR={fpr[optimal_idx]:.5f}")
    
    return optimal_threshold

# For comparison, calculate percentile threshold
percentile_threshold = np.percentile(all_reconstruction_errors, THRESHOLD_PERCENTILE)
print(f"✅ Percentile-based threshold ({THRESHOLD_PERCENTILE}th): {percentile_threshold:.5f}")

# Calculate optimal threshold using ROC curve analysis
optimal_threshold = find_optimal_threshold_roc(true_labels_subset, errors_subset)

# Plot histogram of reconstruction errors with both thresholds
plt.figure(figsize=(12, 6))
plt.hist(all_reconstruction_errors, bins=50, alpha=0.7, density=True)
plt.axvline(percentile_threshold, color='red', linestyle='--', 
            label=f'Percentile Threshold ({THRESHOLD_PERCENTILE}th): {percentile_threshold:.5f}')
plt.axvline(optimal_threshold, color='green', linestyle='--', 
            label=f'Optimal ROC Threshold: {optimal_threshold:.5f}')
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.title('Distribution of Reconstruction Errors with Thresholds')
plt.legend()
plt.grid(True)
plt.savefig('reconstruction_error_histogram.png')

# Compare performance of both thresholds
percentile_preds = (errors_subset > percentile_threshold).astype(int)
optimal_preds = (errors_subset > optimal_threshold).astype(int)

percentile_precision = precision_score(true_labels_subset, percentile_preds, zero_division=0)
percentile_recall = recall_score(true_labels_subset, percentile_preds, zero_division=0)
percentile_f1 = f1_score(true_labels_subset, percentile_preds, zero_division=0)

optimal_precision = precision_score(true_labels_subset, optimal_preds, zero_division=0)
optimal_recall = recall_score(true_labels_subset, optimal_preds, zero_division=0)
optimal_f1 = f1_score(true_labels_subset, optimal_preds, zero_division=0)

print("\n📊 Threshold Method Comparison:")
comparison_df = pd.DataFrame({
    'Method': ['Percentile', 'ROC Optimal'],
    'Threshold': [percentile_threshold, optimal_threshold],
    'Precision': [percentile_precision, optimal_precision],
    'Recall': [percentile_recall, optimal_recall],
    'F1 Score': [percentile_f1, optimal_f1]
})
print(comparison_df)

# Use the optimal threshold for anomaly detection
threshold = optimal_threshold
print(f"\n✅ Using ROC optimal threshold: {threshold:.5f}")

# Apply threshold to get anomaly flags
anomaly_flags_list = [errors > threshold for errors in reconstruction_errors]

# ========================
# 10. EVALUATION
# ========================
# Forecasting metrics
y_pred_all = np.vstack([pred for pred in forecast_list])
y_true_all = np.vstack([true for true in true_windows])
forecast_rmse = np.sqrt(mean_squared_error(y_true_all.reshape(-1), y_pred_all.reshape(-1)))
forecast_mae = mean_absolute_error(y_true_all.reshape(-1), y_pred_all.reshape(-1))
print(f"\n📈 Forecasting Evaluation on Test:")
print(f"RMSE: {forecast_rmse:.5f}")
print(f"MAE:  {forecast_mae:.5f}")

# Anomaly detection metrics - compare both thresholds
all_detected = np.concatenate([flags for flags in anomaly_flags_list])

# Make sure we're only considering windows where we have both predictions and labels
min_length = min(len(simulation_labels), len(all_detected))
true_labels_subset = simulation_labels[:min_length]
all_detected_subset = all_detected[:min_length]

print(f"\nAnomaly detection evaluation using ROC optimal threshold:")
print(f"Array shapes: true_labels={true_labels_subset.shape}, detected={all_detected_subset.shape}")
print(f"Number of true anomalies: {np.sum(true_labels_subset)}")
print(f"Number of detected anomalies: {np.sum(all_detected_subset)}")

# Ensure we have compatible arrays for evaluation metrics
if true_labels_subset.shape != all_detected_subset.shape:
    print(f"⚠️ Warning: Array shapes don't match. Truncating to minimum length.")
    min_len = min(len(true_labels_subset), len(all_detected_subset))
    true_labels_subset = true_labels_subset[:min_len]
    all_detected_subset = all_detected_subset[:min_len]

# Compute metrics using true labels and ROC optimal threshold
precision = precision_score(true_labels_subset, all_detected_subset, zero_division=0)
recall = recall_score(true_labels_subset, all_detected_subset, zero_division=0)
f1 = f1_score(true_labels_subset, all_detected_subset, zero_division=0)

print(f"\n📈 Anomaly Detection Evaluation (ROC Optimal Threshold):")
print(f"Precision: {precision:.5f}")
print(f"Recall:    {recall:.5f}")
print(f"F1 Score:  {f1:.5f}")

# ========================
# 11. SAVE METRICS
# ========================
metrics_results = {
    "Model": "LSTM Seq2Seq + Transformer AE",
    "Threshold_Method": "ROC Optimal",
    "Threshold_Value": threshold,
    "Forecast_RMSE": forecast_rmse,
    "Forecast_MAE": forecast_mae,
    "Anomaly_Precision": precision,
    "Anomaly_Recall": recall,
    "Anomaly_F1": f1
}
metrics_df = pd.DataFrame([metrics_results])
metrics_df.to_csv("metrics_lstm_seq2seq_transformer_ae_pipeline.csv", index=False)
print("\n✅ Metrics saved to 'metrics_lstm_seq2seq_transformer_ae_pipeline.csv'.")

# ========================
# 12. PLOTS
# ========================
# Plot Reconstruction Errors with True Labels
plt.figure(figsize=(14,5))
all_errors = all_reconstruction_errors

# Ensure we're only plotting up to the minimum length we have data for
min_plot_len = min(len(all_errors), len(true_labels_subset))
plot_errors = all_errors[:min_plot_len]
plot_labels = true_labels_subset[:min_plot_len]

plt.plot(plot_errors, label='Reconstruction Error')
plt.axhline(threshold, color='red', linestyle='--', label=f'ROC Optimal Threshold: {threshold:.5f}')

# Detected anomalies
detected_indices = np.where(optimal_preds[:min_plot_len] == 1)[0]
plt.scatter(detected_indices,
            plot_errors[detected_indices],
            color='red', label='Detected Anomalies', s=10)

# True anomalies
true_anomaly_indices = np.where(plot_labels == 1)[0]
if len(true_anomaly_indices) > 0:
    plt.scatter(true_anomaly_indices,
                np.ones_like(true_anomaly_indices) * np.max(plot_errors)*0.9,
                color='green', marker='*', label='True Anomalies', s=20)
else:
    print("No true anomalies found in the subset of data being visualized")

plt.title("Reconstruction Errors vs True Anomalies (ROC Optimal Threshold)")
plt.xlabel("Sample Index")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("reconstruction_errors_lstm_transformer_pipeline.png")
plt.show()